<a href="https://colab.research.google.com/github/ambarja/maplibre-con-python/blob/main/maplibre_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://raw.githubusercontent.com/geografope/maplibre-con-python/main/img/banner.jpg'>
<p align='center'>
<a href='https://www.linkedin.com/in/antonybarja/'><img src='https://img.shields.io/badge/LinkedIn-0077B5?style=for-the-badge&logo=linkedin&logoColor=white'/></a>
<a href='https://www.linkedin.com/in/antonybarja/'><img src='https://img.shields.io/badge/TikTok-000000?style=for-the-badge&logo=tiktok&logoColor=white'/></a>
<a href=" https://www.youtube.com/@geografope"><img src="https://img.shields.io/badge/youtube-%23FF0000.svg?&style=for-the-badge&logo=youtube&logoColor=white"></a>
<a href='https://github.com/geografope'><img src='https://img.shields.io/badge/GitHub-100000?style=for-the-badge&logo=github&logoColor=white' /></a>
<a href="https://open.spotify.com/user/31rnma4md2l7hw77z5rynhrzbvqm"><img src="https://img.shields.io/badge/Spotify-1ED760?&style=for-the-badge&logo=spotify&logoColor=white"></a>
<a href="https://twitter.com/antony_barja"><img src="https://img.shields.io/badge/Twitter-1DA1F2?style=for-the-badge&logo=twitter&logoColor=white"></a>
</p>

### **1. Instalación de librerías**

In [11]:
!pip install maplibre
!pip install geopandas

### **2. Activación o llamado de librerías**

In [5]:
from maplibre import Layer, LayerType, Map, MapOptions
from maplibre.controls import FullscreenControl,ScaleControl
from maplibre.ipywidget import MapWidget as Map
from maplibre.sources import GeoJSONSource,RasterTileSource
from maplibre.basemaps import Carto
from maplibre.utils import geopandas_to_geojson
import geopandas as gpd
import ipywidgets as widgets

### **3. Hola mundo con MapLibre**

In [7]:
m = Map(MapOptions(style = 'https://demotiles.maplibre.org/style.json'))
m

MapWidget(height='400px', map_options={'style': 'https://demotiles.maplibre.org/style.json'})

### **4. Cambiar el BaseMap y agregar Widget**
Actualmente por defecto se tiene los vector ttiles de los estilos de **Carto**, entre sus tipos tenemos a:

- DARK_MATTER
- POSITRON
- VOYAGER
- POSITRON_NO_LABELS
- DARK_MATTER_NOLABELS
- VOYAGER_NOLABELS

Pero si deseas personalizar los vector tiles o deseas cambiar otros servicios de mapas base,
puedes visitar a MapTiler : https://cloud.maptiler.com/maps/

In [9]:
m = Map(MapOptions(style = Carto.VOYAGER_NOLABELS))
m

MapWidget(height='400px', map_options={'style': 'https://basemaps.cartocdn.com/gl/voyager-nolabels-gl-style/st…

In [10]:
# Estilos de mapabase
osm = 'https://api.maptiler.com/maps/openstreetmap/style.json?key=o1dsIFoKyJkSQWPNCrvz'
sat = 'https://api.maptiler.com/maps/hybrid/style.json?key=o1dsIFoKyJkSQWPNCrvz'
blue = 'https://api.maptiler.com/maps/9133a174-4328-45fc-89fa-d6f34d82f802/style.json?key=o1dsIFoKyJkSQWPNCrvz'

bm_osm = Map(MapOptions(style = osm))
bm_sat = Map(MapOptions(style = sat))
bm_carto = Map(MapOptions(style = Carto.VOYAGER))
bm_blue = Map(MapOptions(style = blue))

In [11]:
cuadricula = widgets.GridBox([bm_osm, bm_sat, bm_carto, bm_blue], layout=widgets.Layout(grid_template_columns="50% 50%"))
display(cuadricula)

GridBox(children=(MapWidget(height='400px', map_options={'style': 'https://api.maptiler.com/maps/openstreetmap…

### **5. Integración con Geopandas**

In [12]:
# Opendata: https://github.com/ambarja/gpkg-pe
import pandas as pd
url = 'https://raw.githubusercontent.com/ambarja/snowfalldb-peru/main/csv/snowfallDB-Peru.csv'
nevada_data = pd.read_csv(url)
nevada_data = gpd.GeoDataFrame(data = nevada_data, geometry=gpd.points_from_xy(nevada_data.lon, nevada_data.lat),crs = 4326).dropna()

In [13]:
nevada_data.head()

,CDpto,Dep,CProv,Prov,CDist,Dist,Codigo,Fenomeno,Afectado,Fecha,lon,lat,geometry
0,3.0,APURIMAC,301.0,ABANCAY,30101.0,ABANCAY,59933.0,PRECIPITACIONES - NEVADA,250.0,28/08/2013,-72.8793,-13.6368,POINT (-72.87930 -13.63680)
1,3.0,APURIMAC,301.0,ABANCAY,30102.0,CHACOCHE,59992.0,PRECIPITACIONES - NEVADA,0.0,28/08/2013,-72.9930,-13.9422,POINT (-72.99300 -13.94220)
2,3.0,APURIMAC,301.0,ABANCAY,30102.0,CHACOCHE,71863.0,PRECIPITACIONES - NEVADA,840.0,14/07/2015,-72.9930,-13.9422,POINT (-72.99300 -13.94220)
3,3.0,APURIMAC,301.0,ABANCAY,30103.0,CIRCA,59978.0,PRECIPITACIONES - NEVADA,900.0,28/08/2013,-72.8788,-13.8790,POINT (-72.87880 -13.87900)
4,3.0,APURIMAC,301.0,ABANCAY,30103.0,CIRCA,71935.0,PRECIPITACIONES - NEVADA,482.0,8/7/2015,-72.8788,-13.8790,POINT (-72.87880 -13.87900)


In [14]:
region_source = GeoJSONSource(data=geopandas_to_geojson(nevada_data))

In [16]:
nevadas = Layer(
    type='heatmap',
    id='nevada_id',
    source=region_source,
    paint={
        "heatmap-weight":  ["interpolate", ["linear"],["get","Afectados"],0,0,6,1],
        "heatmap-radius": ["interpolate", ["linear"],["zoom"],0,2,9,10],
        "heatmap-color": ["interpolate", ["linear"],["heatmap-density"],0,'rgba(33,102,172,0)',0.2,'rgb(103,169,207)',0.4,'rgb(209,229,240)',0.6,'rgb(253,219,199)',0.8,'rgb(239,138,98)',1,'rgb(178,24,43)']
    },
)

m = Map(MapOptions(style = osm,bounds=nevada_data.total_bounds))
m.add_layer(nevadas)
m

MapWidget(calls=[['addLayer', ({'id': 'nevada_id', 'type': 'heatmap', 'paint': {'heatmap-weight': ['interpolat…

### **6. Agregar servicos de mapas web**

In [17]:
SOURCE_ID = 'cenepred_id'
LAYER_ID = 'inundaciones'
raster_layer = 'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/af53044acb3cf0bd140b6b629e84e89e-5d96056e54bdc99708a967a0d2eab445/tiles/{z}/{x}/{y}'
wms_source = RasterTileSource(
    tiles = [raster_layer],
    tile_size = 256
)
wms_layer = Layer(type = LayerType.RASTER,source =SOURCE_ID, id=LAYER_ID)
m = Map(map_options=MapOptions(zoom = 4,center = (-72.99564,-8.49248)))
m.use_message_queue()
m.add_source(SOURCE_ID,wms_source)
m.add_layer(wms_layer)
m.add_control(FullscreenControl())
m

MapWidget(height='400px', map_options={'center': (-72.99564, -8.49248), 'style': 'https://basemaps.cartocdn.co…

### **7. Exportación de resultados**

In [18]:
with open("index.html", "w") as f:
    f.write(m.to_html(style="height:800px;"))

/usr/local/lib/python3.10/dist-packages/maplibre
